# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:  
        csvreader = csv.reader(csvfile) 
        next(csvreader)
               
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster and establishing connection

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
 try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1} """
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### QUERY 1: 
##### 1. The requirement is to query artist, song_title and song's length based on session_id and itemInSession so we created a composite primary key with partition key as sessionId and clustering column as itemInSession. 
##### 2. The table description is as follows:
               TABLE NAME: song_in_session 
                   session_id int
                   itemInSession int
                   artist_name text
                   song_title text
                   song_length decimal
                   PRIMARY KEY(session_id, itemInSession)

In [8]:
try:
    session.execute("""CREATE TABLE IF NOT EXISTS song_in_session (
                       session_id int, itemInSession int, artist_name text, song_title text, song_length decimal,
                       PRIMARY KEY(session_id, itemInSession))""")
except Exception as e:
    print(e)        

#### The following code reads the csv file and inserts the data into 'song_in_session' table

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_in_session(session_id, itemInSession, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[-3]), int(line[3]), line[0], line[-2], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "SELECT artist_name, song_title, song_length FROM song_in_session \
         WHERE session_id = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### QUERY 2: 
##### 1. The requirement is to query artist, song_title sorted by itemInSession, user's first_name and last_name based on user_id and session_id so we created a composite primary key with partition key as user_id and sessionId and clustering column as itemInSession. 
##### 2. The reason to use user_id and session_id both as the partition key and not just user_id is because we want session_id's that belong to the same user will be put on different tables which can result in performance issues if the volume of the data increases.
##### 3. Finally we use itemInSession as the clustering column because we want data to be sorted based on this column.
##### 4. The table description is as follows:
               TABLE NAME: user_session_details 
                   user_id int
                   session_id int
                   artist_name text
                   song_title text
                   itemInSession int
                   first_name text
                   last_name text
                   PRIMARY KEY((user_id, session_id), itemInSession)

In [11]:
try:
    session.execute("""
        CREATE TABLE user_session_details(user_id int, session_id int,artist_name text, song_title text, 
                     itemInSession int, first_name text, last_name text,
                     PRIMARY KEY((user_id, session_id), itemInSession))"""
    )
except Exception as e:
    print(e)

                    

#### The following code reads the csv file and inserts the data into 'song_in_session' table

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_details(user_id, session_id, artist_name, song_title, itemInSession, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[-1]), int(line[-3]), line[0], line[-2], int(line[3]), line[1], line[4]))

#### SELECT to verify that the data have been inserted into each table

In [13]:
query = "SELECT itemInSession, artist_name, song_title, first_name, last_name FROM user_session_details WHERE user_id=10 AND session_id=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.iteminsession, row.artist_name, row.song_title, row.first_name, row.last_name)

0 Down To The Bone Keep On Keepin' On Sylvie Cruz
1 Three Drives Greece 2000 Sylvie Cruz
2 Sebastien Tellier Kilometer Sylvie Cruz
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### QUERY 3: 
##### 1. The requirement is to query every user's first_name and last_name based on song_title so we created a composite primary key with partition key as song_title and clustering column as user_id.
##### 2. The reason to select user_id as clustering column is that multiple users are likely to listen to a common song. If clustering column is not added only 1 entry for a song_title is inserted in the database. We want to preserve the user information to display all the users listening to a particular song.
##### 3. user_id is used to uniquely identify a user, so we add clustering column as user_id.
##### 4. The table description is as follows:
               TABLE NAME: user_songs
                   song_title text
                   user_id int
                   first_name text
                   last_name text
                   PRIMARY KEY(song_title, user_id)

In [14]:
try:
    session.execute("""
    CREATE TABLE user_songs(song_title text, user_id int, first_name text, last_name text, 
                 PRIMARY KEY(song_title, user_id))"""
    )
except Exception as e:
    print(e)

#### The following code reads the csv file and inserts the data into 'song_in_session' table

In [15]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs(user_id, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[-1]), line[-2], line[1], line[4]))

#### SELECT to verify that the data have been inserted into each table

In [16]:
query = "SELECT first_name, last_name FROM user_songs WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name,row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
try:
    session.execute("DROP TABLE IF EXISTS song_in_session")
    session.execute("DROP TABLE IF EXISTS user_session_details")
    session.execute("DROP TABLE IF EXISTS user_songs")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()